### Notice!!! Please use search function to check all the cells with "notice" keywords, those are sanity checks and potential manual variable changes

In [0]:
#########################################################################################################
# Please define the target Timestamp for the end date for the generation of COVID patients dataset
# Notice: Format: YYYY-MM-DD
# Current set to 2021-12-10
############################################################################################################
# check to make sure they are consistent
# Previous dates used:
# 1. "2021-12-31", "20211231"
# 2. "2022-02-18", "20220218"
# 3. end_date, file_date = "2021-12-25", "20211225"
# end_date, file_date = "2021-12-21", "20211221"

start_date, end_date, file_date = "2021-12-26", "2022-08-30", "20211226_20220830_Lancet_bmi"
# register it so can be directly used in the following SQL statement
spark.conf.set("enddate.var", end_date)

In [0]:
#pip install xgboost
#pip install joblib
#pip install scikit-learn
#pip install shap

### Keep the follow cell empty can copy all the install commands and run them one by one

In [0]:
import numpy as np
import pandas as pd
import shap

## Scikit-learn related
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.linear_model import SGDClassifier
# from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.ensemble import BaggingClassifier

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
## from sklearn import cross_validation, metrics   #Additional scklearn functions, deprecated since version 0.18
## from sklearn.grid_search import GridSearchCV   #!!!the grid search package that has issue, dont use it
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from scipy import interp
from scipy import stats

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing

########################################
## Import those from pyspark.ml
import pyspark.ml
import pyspark.sql.functions as F

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-

In [0]:
imids_training_df = spark.sql("""SELECT * FROM rdp_phi_sandbox.qw_IMID_COVID_trainset_cond_med_vax_{}""".format(file_date)) 
# imids_training_df.limit(5).toPandas()

In [0]:
## Read paxlovid patient cohort
pts_paxlovid_df = spark.sql("""SELECT * FROM rdp_phi_sandbox.qw_IMID_COVID_trainset_Omicron_Paxlovid_{}""".format(file_date))
## Innere join to get the column of paxlovid

imids_training_df = imids_training_df.join(pts_paxlovid_df, imids_training_df.pat_id ==  pts_paxlovid_df.pat_id, "left").fillna(0).drop(pts_paxlovid_df.pat_id)

In [0]:
imids_training_df = imids_training_df.dropDuplicates()
# imids_training_df.limit(5).toPandas()

In [0]:
## Drop not used columns
col_to_drop = ('age_range', 'ethnicity', 'race1', 'race_v2', 'ethnicity_race', 'CVX_name', 'decided_index_date', 'CVX_name')
imids_training_df = imids_training_df.drop(*col_to_drop)

## rename IMIDs drugs columns
imids_training_df = imids_training_df.withColumnRenamed('prior_91_days_hydroxychloroquine_logic', 'hydroxychloroquine').withColumnRenamed('prior_91_days_methotrexate_logic', 'methotrexate')\
.withColumnRenamed('prior_91_days_leflunomide_teriflunomide_logic', 'leflunomide_teriflunomide').withColumnRenamed('prior_91_days_5_ASAa_logic', '5_ASA')\
.withColumnRenamed('prior_91_days_azathioprine_logic', 'azathioprine').withColumnRenamed('prior_91_days_mercaptopurine_logic', 'mercaptopurine')\
.withColumnRenamed('prior_91_days_mitoxantrone_logic', 'mitoxantrone')\
.withColumnRenamed('prior_91_days_mycophenolate_logic', 'mycophenolate').withColumnRenamed('prior_91_days_calcineurin_inhibitor_logic', 'calcineurin_inhibitor')\
.withColumnRenamed('prior_91_days_TNF_alpha_inhibitor_logic', 'TNF_alpha_inhibitor').withColumnRenamed('prior_91_days_fumarates_logic', 'fumarates')\
.withColumnRenamed('prior_91_days_interferons_logic', 'interferons').withColumnRenamed('prior_91_days_alkylating_agent_logic', 'alkylating_agent')\
.withColumnRenamed('prior_91_days_hydroxyurea_logic', 'hydroxyurea').withColumnRenamed('prior_91_days_dapsone_logic', 'dapsone')\
.withColumnRenamed('prior_91_days_cladribine_logic', 'cladribine').withColumnRenamed('prior_91_days_IL1_inhibitor_logic', 'IL1_inhibitor')\
.withColumnRenamed('prior_91_days_IL6_inhibitor_logic', 'IL6_inhibitor').withColumnRenamed('prior_91_days_IL12_23_inhibitor_logic', 'IL12_23_inhibitor')\
.withColumnRenamed('prior_91_days_IL17_inhibitor_logic', 'IL17_inhibitor')\
.withColumnRenamed('prior_91_days_IL23_inhibitor_logic', 'IL23_inhibitor')\
.withColumnRenamed('prior_91_days_abatacept_logic', 'abatacept').withColumnRenamed('prior_91_days_anti_BLyS_logic', 'anti_BLyS')\
.withColumnRenamed('prior_91_days_S1P_receptor_modulator_logic', 'S1P_receptor_modulator').withColumnRenamed('prior_91_days_JAK_inhibitor_logic', 'JAK_inhibitor')\
.withColumnRenamed('prior_91_days_integrin_inhibitor_logic', 'integrin_inhibitor').withColumnRenamed('prior_91_days_PDE4i_targeted_synthetic_logic', 'PDE4i_targeted_synthetic')\
.withColumnRenamed('prior_91_days_anti_CD20_logic', 'anti_CD20').withColumnRenamed('prior_91_days_anti_CD52_logic', 'anti_CD52')\
.withColumnRenamed('prior_91_days_budesonide_logic', 'budesonide').withColumnRenamed('prior_91_days_systemic_glucocorticoids_logic', 'systemic_glucocorticoids')\
.withColumnRenamed('after_10_days_monoclonal_antibody_covid_19_logic', 'monoclonal_antibody_covid_19')\
.withColumnRenamed('after_14_days_paxlovid_logic', 'paxlovid')

In [0]:
## Temp solution, filter the positive patient here
pos_df = imids_training_df.filter(imids_training_df['results'] == "Positive")

## convert to pandas dataframe
# imids_training_pd_df = imids_training_onlyPos_df.toPandas()
 
## col names before dropping any
# list(imids_training_pd_df.columns)

In [0]:
## One hot-encode sex
categ = pos_df.select('sex').distinct().rdd.flatMap(lambda x:x).collect()
exprs = [F.when(F.col('sex') == cat,1).otherwise(0)\
            .alias(str("sex_" + cat)) for cat in categ]
pos_one_hot_df = pos_df.select(pos_df.columns + exprs)

## Fill none values in the vaccination column
pos_one_hot_df = pos_one_hot_df.fillna({'Vaccination_status':'Not'})

## One hot-encode vaccination
categ = pos_one_hot_df.select('Vaccination_status').distinct().rdd.flatMap(lambda x:x).collect()
exprs = [F.when(F.col('Vaccination_status') == cat,1).otherwise(0)\
            .alias(str("vaccination_" + cat)) for cat in categ]
pos_one_hot_df = pos_one_hot_df.select(pos_one_hot_df.columns + exprs)

## Drop not needed cols and original cols
pos_one_hot_df = pos_one_hot_df.drop("sex").drop("sex_Female").drop("sex_Unknown").drop("Vaccination_status").drop("vaccination_Not")

## change the name of those vaccination related columns
pos_one_hot_df = pos_one_hot_df.withColumnRenamed('vaccination_Fully', 'fully_vaccinated').withColumnRenamed('vaccination_Booster', 'boosted')

In [0]:
# ## Manually define the min max process
# from pyspark.sql.functions import max, min, mean, col

# max_age, min_age = pos_one_hot_df.select(max("age"), min("age")).first()
# pos_one_hot_minmax_df = pos_one_hot_df.withColumn("age_normalized", (col("age") - min_age) / (max_age - min_age) )
# pos_one_hot_minmax_df = pos_one_hot_minmax_df.drop("age")

In [0]:
# ## get the data where monoclonal_antibody_covid_19 = 0
# pos_one_hot_noAntibody_df = pos_one_hot_df.where(pos_one_hot_df.monoclonal_antibody_covid_19 == 0)

In [0]:
train_df = pos_one_hot_df.select("*")
train_df = train_df.drop("pat_id").drop("results")

## Drop SVI and geocoding features due to high missing percentage
## list: 'SVI_Socioeconomic', 'SVI_Household_Composition_Disability', 'SVI_Minority_Status_Language', 'SVI_Housing_Type_Transportation', 'SVI', 'Metro_area', 'Low_education', 'Low_employment',
## not including 'obesity'

train_df = train_df.select('patient_id', 'hospitalized_after_positive','IMV_after_positive','death_after_positive',
  'age', 'BMI', 'sex_Male',
  'hypertension', 'diabetes_type1and2', 'atrial_fibrillation', 'coronary_artery_disease', 'heart_failure', 'chronic_kidney_disease', 'copd', 'chronic_liver_disease', 'malignant_neoplastic_disease', 'HIV', 'history_transplant', 'stroke', 'opioid_dependence', 'fully_vaccinated', 'boosted', 'asthma', 
 'ibd', 'rheumatoid_arthritis', 'multiple_sclerosis','psoriatic_arthritis', 'psoriasis', 'systemic_sclerosis', 'spondyloarthritis', 'systemic_lupus', 'vasculitis', 'sarcoidosis', 'APS', 'sjogren_syndrome',
 'hydroxychloroquine',
 'methotrexate',
 'leflunomide_teriflunomide',
 '5_ASA',
 'azathioprine',
 'mercaptopurine',
 'mitoxantrone',
 'mycophenolate',
 'calcineurin_inhibitor',
 'TNF_alpha_inhibitor',
 'fumarates',
 'interferons',
 'alkylating_agent',
 'hydroxyurea',
 'dapsone',
 'cladribine',
 'IL1_inhibitor',
 'IL6_inhibitor',
 'IL12_23_inhibitor',
 'IL17_inhibitor',
 'IL23_inhibitor',
 'abatacept',
 'anti_BLyS',
 'S1P_receptor_modulator',
 'JAK_inhibitor',
 'integrin_inhibitor',
 'PDE4i_targeted_synthetic',
 'anti_CD20',
 'anti_CD52',
 'budesonide',
 'systemic_glucocorticoids',
 'monoclonal_antibody_covid_19',
 'paxlovid')

In [0]:
train_pd_df = train_df.toPandas()
## change the patient id col to be the index of the training data
train_pd_df = train_pd_df.set_index("patient_id")
# display(train_pd_df)

## Training data preprocessing and test data split
* Assign features for training data to X
* Assign response to Y
* Define the number of CV & random seed used for tuning

In [0]:
# ### Uncomment to save table as backup for publications
# spark.sql("""DROP TABLE IF EXISTS rdp_phi_sandbox.qw_IMIDs_COVID_paper_train_data_processed_{}""".format(file_date))
# table_name = "rdp_phi_sandbox.qw_IMIDs_COVID_paper_train_data_processed_{}".format(file_date)
# ## Convert into Spark DataFrame
# spark_df_one_hot_encoded = spark.createDataFrame(df_one_hot_encoded)

# ## Write the table
# # spark_df_one_hot_encoded.write.mode("overwrite").saveAsTable(table_name)

In [0]:
#######################################################
#Random search CV method
#and
#Multi class roc_auc score method
########################################################
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from time import time
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer, roc_auc_score

###############################################################################################
#Binary class roc auc score method
#input: y_true, true labels from test fold
#       y_score, predicted probability on test fold
#       average, string, [None, ‘micro’, ‘macro’ (default), ‘samples’, ‘weighted’]
#                'macro': Calculate metrics for each label, and find their unweighted mean. 
#                This does not take label imbalance into account.
#                'weighted': Calculate metrics for each label, and find their average, 
#                weighted by support (the number of true instances for each label).
#output: auroc value for each class
#############################################################################################
def binary_class_roc_auc_score(y_true, y_score, average="weighted"):

    return roc_auc_score(y_true, y_score, average=average)

binaryclass_score = make_scorer(binary_class_roc_auc_score, needs_threshold = True)

In [0]:
## not including: , 'obesity'
train_pd_df.columns = ['hospitalized_after_positive', 'IMV_after_positive', 'death_after_positive', 'age', 'BMI', 'sex:male', 'hypertension', 'diabetes (type 1+2)', 'atrial fibrillation', 'coronary artery disease', 'heart failure', 'chronic kidney disease', 'COPD', 'chronic liver disease', 'malignant neoplastic disease', 'HIV', 'history of transplant', 'stroke', 'opioid dependence', 'fully vaccinated', 'boosted', 'asthma',
 'inflammatory bowel disease', 'rheumatoid arthritis', 'multiple sclerosis','psoriatic arthritis', 'psoriasis', 'systemic sclerosis', 'spondyloarthritis', 'systemic lupus', 'vasculitis', 'sarcoidosis', 'antiphospholipid syndrome', 'Sjögren syndrome',
 'hydroxychloroquine',
 'methotrexate',
 'leflunomide teriflunomide',
 '5-ASA',
 'azathioprine',
 'mercaptopurine',
 'mitoxantrone',
 'mycophenolate',
 'calcineurin inhibitor',
 'TNF-α inhibitor',
 'fumarates',
 'interferons',
 'alkylating agent',
 'hydroxyurea',
 'dapsone',
 'cladribine',
 'IL-1 inhibitor',
 'IL-6 inhibitor',
 'IL-12/23 inhibitor',
 'IL-17 inhibitor',
 'IL-23 inhibitor',
 'abatacept',
 'anti-BLyS',
 'S1P receptor modulator',
 'JAK inhibitor',
 'integrin inhibitor',
 'PDE4i targeted synthetic',
 'anti-CD20',
 'anti-CD52',
 'budesonide',
 'systemic glucocorticoids', 'monoclonal antibody covid-19', 'paxlovid']

In [0]:
## Columns to drop based on VIF results or missingness
# train_pd_df.drop(['HIV'], axis = 1, inplace = True)
# train_pd_df.drop(['systemic sclerosis'], axis = 1, inplace = True)
# train_pd_df.drop(['leflunomide teriflunomide'], axis = 1, inplace = True)
# train_pd_df.drop(['5-ASA'], axis = 1, inplace = True)
train_pd_df.drop(['mitoxantrone'], axis = 1, inplace = True)
# train_pd_df.drop(['mycophenolate'], axis = 1, inplace = True)
# train_pd_df.drop(['interferons'], axis = 1, inplace = True)
# train_pd_df.drop(['alkylating agent'], axis = 1, inplace = True)
# train_pd_df.drop(['hydroxyurea'], axis = 1, inplace = True)

# train_pd_df.drop(['dapsone'], axis = 1, inplace = True)
train_pd_df.drop(['cladribine'], axis = 1, inplace = True)
train_pd_df.drop(['IL-1 inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['IL-6 inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['IL-23 inhibitor'], axis = 1, inplace = True)
train_pd_df.drop(['anti-BLyS'], axis = 1, inplace = True)
# train_pd_df.drop(['S1P receptor modulator'], axis = 1, inplace = True)
train_pd_df.drop(['integrin inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['PDE4i targeted synthetic'], axis = 1, inplace = True)
# train_pd_df.drop(['anti-CD20'], axis = 1, inplace = True)
train_pd_df.drop(['anti-CD52'], axis = 1, inplace = True)

# train_pd_df.drop(['mercaptopurine'], axis = 1, inplace = True)
# train_pd_df.drop(['abatacept'], axis = 1, inplace = True)

# train_pd_df.drop(['IL-1 inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['cladribine'], axis = 1, inplace = True)
# train_pd_df.drop(['anti-CD52'], axis = 1, inplace = True)
# train_pd_df.drop(['monoclonal antibody covid-19'], axis = 1, inplace = True)


percent_missing = train_pd_df.isnull().sum() * 100 / len(train_pd_df)
missing_value_df = pd.DataFrame({'column_name': train_pd_df.columns,
                                 'percent_missing': percent_missing})

display(missing_value_df)

## Notice! Need to comment out once have the initial VIF results
## Fill NA values
## Method 1: Remove rows with at least one null value
# train_pd_noNull_df = train_pd_df.dropna()
# train_pd_df = train_pd_noNull_df

## Notice
## Remove mAbs, antibody feature here
train_pd_df.drop(['monoclonal antibody covid-19'], axis = 1, inplace = True)

column_name,percent_missing
hospitalized_after_positive,0.0
IMV_after_positive,0.0
death_after_positive,0.0
age,0.0
BMI,0.0
sex:male,0.0
hypertension,0.0
diabetes (type 1+2),0.0
atrial fibrillation,0.0
coronary artery disease,0.0


In [0]:
## manually set the random seed to define a replication
r_seed = 42
print("current random seed is: ", r_seed)

## manually set the number for cross validation
num_cv = 10
print("current CV fold selection is: ", num_cv)

## Possible response vectors
Y_cols = ["hospitalized_after_positive", 'IMV_after_positive', 'death_after_positive']

## separate X and Y
train_df_Y = train_pd_df[Y_cols]
train_df_X = train_pd_df.drop(Y_cols, axis=1)

## Create composite response vectors
train_df_Y_new = pd.DataFrame(train_df_Y, columns = Y_cols)

train_df_Y_new['hospitalized_or_IMV_or_death'] = train_df_Y['hospitalized_after_positive'] + train_df_Y['IMV_after_positive'] + train_df_Y['death_after_positive']
train_df_Y_new['IMV_or_death'] = train_df_Y['IMV_after_positive'] + train_df_Y['death_after_positive']

## Convert those >1 values back to 1
train_df_Y_new.loc[train_df_Y_new['hospitalized_or_IMV_or_death'] >= 1, 'hospitalized_or_IMV_or_death'] = 1
train_df_Y_new.loc[train_df_Y_new['IMV_or_death'] >= 1, 'IMV_or_death'] = 1

current random seed is:  42
current CV fold selection is:  10


In [0]:
## Notice: LR need a minimum of 10 events (>50 better) per independent variable
## Therefore, the following features need to be excluded for lack of events

# train_df_X.iloc[:,2:].apply(pd.Series.value_counts)

In [0]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = train_df_X.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(train_df_X.values, i) for i in range(len(train_df_X.columns))]

display(vif_data)

feature,VIF
age,3.5818556333315725
BMI,1.0198399442534696
sex:male,1.567038652109814
hypertension,1.8732369908794655
diabetes (type 1+2),1.37731399749496
atrial fibrillation,1.354896743157273
coronary artery disease,1.3535349473415597
heart failure,1.503543745492051
chronic kidney disease,1.410116063894732
COPD,1.2134800652081756


In [0]:
# import matplotlib.pyplot as plt
# from bioinfokit import visuz

# visuz.stat.corr_mat(df=train_df_X, cmap='RdBu',dim = (15,15),  show=True)

In [0]:
## Found workaround at blog: this https://stackoverflow.com/questions/71106940/cannot-import-name-centered-from-scipy-signal-signaltools
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

In [0]:
import statsmodels.api as sm
import statsmodels
import lxml

## The regularization method AND the solver used is determined by the argument method.
regular_method = "l1"

##Set LR optimzer to be
## Options: lbfgs, powell, cg, ncg, bfgs, basinhopping, newton, ‘nm’ for Nelder-Mead
## Previous choice: ncg
opt_method = 'lbfgs'

####################################################################################################
## Method used to do univariate logistic regression on each single feature/variable
## Inputs:
## os_X: oversampled training data set X
## os_y: oversampled training data's label y
## X: training data set X
## y: training data's label y
## variable_name: the current feature/variable to use for the univariate LR model
## opt_method: the current approximation method used for the LR model
## Output:
## A dataframe with columns: Feature, Feature Importance, Odds ratio, Pvalue, lower, upper
######################################################################################################
def multivariate_LR_model_result(os_X, os_y, us_X, us_y, select_method):
  ## Instantiate a bionomial family and logit link function GLM model as logistic regression model
  os_model = sm.Logit(os_y, os_X, missing = 'drop').fit(method = select_method, skip_hessian = False)

  print(os_model.summary())

  ## Acquire LR summary
  os_results_summary = os_model.summary()
  ## Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
  os_results_as_html = os_results_summary.tables[1].as_html()
  os_odds_ratio_df = pd.read_html(os_results_as_html, header=0, index_col=0)[0]

    ## get the CI interval
  os_odds_ratio_df = os_odds_ratio_df[["coef", "[0.025", "0.975]"]]
  os_odds_ratio_df.rename(columns={'coef': 'Odds ratio', '[0.025': 'lower', '0.975]':'upper'}, inplace=True)

  ## Instantiate a bionomial family and logit link function GLM model as logistic regression model
  us_model = sm.Logit(us_y, us_X, missing = 'drop').fit(method = select_method, skip_hessian = False)

  #print(us_model.summary())

  ## Acquire LR summary
  us_results_summary = us_model.summary()
  ## Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
  us_results_as_html = us_results_summary.tables[1].as_html()
  us_odds_ratio_df = pd.read_html(us_results_as_html, header=0, index_col=0)[0]

  ## Get the uncorrected P value
  us_odds_ratio_df = us_odds_ratio_df[["P>|z|"]]
  us_odds_ratio_df.rename(columns={'P>|z|': 'Pvalue'}, inplace=True)
  
  ## Get the corrected P value
  ## Now use bonferroni
  ## check this document for more details: http://jpktd.blogspot.com/2013/04/multiple-testing-p-value-corrections-in.html
  ## API: https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html
  ## Options: conservative Bonferroni correction method
  rej, pval_corr= statsmodels.stats.multitest.multipletests(us_odds_ratio_df['Pvalue'], alpha=0.1, method='fdr_by', is_sorted=False)[:2]
  us_odds_ratio_df['Pvalue_FDR_corrected'] = pval_corr

  odds_ratio_df = pd.concat([os_odds_ratio_df.reset_index(drop=True), us_odds_ratio_df.reset_index(drop=True)], axis=1)

  ## Display the odds ratio, p value, feature importance dataframe
  feature_importances = pd.DataFrame(os_model.conf_int()[1]).rename(columns={1:'Coefficients'}).eval("absolute_coefficients=abs(Coefficients)")
  feature_importances.reset_index(inplace=True)

  ## if feature importance is needed
  feature_importances_df = feature_importances[["index", "absolute_coefficients"]]
  feature_importances_df.rename(columns={'index': 'Feature', 'absolute_coefficients': 'Feature Importance'}, inplace=True)
  ## if feature importance is not needed
#   feature_importances_df = feature_importances[["index"]]
#   feature_importances_df.rename(columns={'index': 'Feature'}, inplace=True)

  result_df = pd.concat([feature_importances_df.reset_index(drop=True), odds_ratio_df.reset_index(drop=True)], axis=1)
  return result_df, os_model

## Response == "death"

In [0]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

###############################################################################
## Current Y as response
## Notice: need manual check
## Possible options: hospitalized, invasive_mechanical_vent, death, results
################################################################################

select_col = 'death_after_positive'
Y = train_df_Y[select_col]
# Y = Y.map(dict(yes=1, no=0))

class_names = np.unique(Y)
print("unique labels from y: ", class_names)

## Train test split use r_seed assigned in CMD 1
X = train_df_X
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=r_seed, stratify = Y)
columns = X_train.columns

## Normalize after the train/test split to avoid any potential data leakage
## using the min and max from the trainset to do minmax on test set
age_min, age_max = X_train["age"].min(), X_train["age"].max()
X_train["age"] = (X_train["age"] - age_min) / (age_max - age_min)
X_test["age"] = (X_test["age"] - age_min) / (age_max - age_min)

BMI_min, BMI_max = X_train["BMI"].min(), X_train["BMI"].max()
X_train["BMI"] = (X_train["BMI"] - BMI_min) / (BMI_max - BMI_min)
X_test["BMI"] = (X_test["BMI"] - BMI_min) / (BMI_max - BMI_min)

## Random oversampling using random seed
## Define oversampling strategy: default, 'auto': equivalent to 'not majority'
oversample = RandomOverSampler(random_state=r_seed)
os_data_X, os_data_y = oversample.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X, columns=columns)
os_data_y= pd.DataFrame(data=os_data_y, columns=['death_after_positive'])


# define undersample strategy: “majority” will undersample the majority class determined by the class with the largest number of examples.
# undersample = RandomUnderSampler(sampling_strategy='majority', random_state=r_seed)
# us_data_X, us_data_y = undersample.fit_resample(X_train, y_train)
# us_data_X = pd.DataFrame(data=us_data_X, columns=columns)
# us_data_y= pd.DataFrame(data=us_data_y, columns=['death_after_positive'])


# we can Check the numbers of our data
# print("length of oversampled data is ",len(os_data_X))
# print("Number of no hos in oversampled data",len(os_data_y[os_data_y['hospitalized_after_positive']==0]))
# print("Number of hos",len(os_data_y[os_data_y['hospitalized_after_positive']==1]))
# print("Proportion of no hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==0])/len(os_data_X))
# print("Proportion of hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==1])/len(os_data_X))

unique labels from y:  [0 1]


In [0]:
death_result_df, os_LR_statmodel = multivariate_LR_model_result(os_data_X, os_data_y, X_train, y_train, opt_method)
display(death_result_df)

Maximum Likelihood optimization failed to converge. Check mle_retvals


                            Logit Regression Results                            
Dep. Variable:     death_after_positive   No. Observations:               213948
Model:                            Logit   Df Residuals:                   213891
Method:                             MLE   Df Model:                           56
Date:                  Wed, 03 May 2023   Pseudo R-squ.:                  0.1926
Time:                          00:02:45   Log-Likelihood:            -1.1974e+05
converged:                        False   LL-Null:                   -1.4830e+05
Covariance Type:              nonrobust   LLR p-value:                     0.000
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
age                              0.8845      0.016     54.373      0.000       0.853       0.916
BMI                             -0.7643      1.352     -0.565

Maximum Likelihood optimization failed to converge. Check mle_retvals

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Feature,Feature Importance,Odds ratio,lower,upper,Pvalue,Pvalue_FDR_corrected
age,0.9163462226000652,0.8845,0.853,0.916,0.0,0.0
BMI,1.8853721387360847,-0.7643,-3.414,1.885,0.382,1.0
sex:male,0.41005626626438835,-0.4291,-0.448,-0.41,0.0,0.0
hypertension,0.06302357189573903,-0.0882,-0.113,-0.063,0.0,0.0
diabetes (type 1+2),0.1484449595271243,0.1189,0.089,0.148,0.232,1.0
atrial fibrillation,0.8187226214373883,0.7843,0.75,0.819,0.0,0.0
coronary artery disease,0.414655987497526,0.3793,0.344,0.415,0.0,0.0
heart failure,0.9677653117771767,0.9324,0.897,0.968,0.0,0.0
chronic kidney disease,0.8285629885769543,0.7963,0.764,0.829,0.0,0.0
COPD,0.7296121089443863,0.6915,0.653,0.73,0.0,0.0


In [0]:
from sklearn.metrics import roc_auc_score
LR_model_binaryclass_auroc = binary_class_roc_auc_score(y_test, os_LR_statmodel.predict(X_test))

print("Auroc on test data set: %0.5f" % (LR_model_binaryclass_auroc))

Auroc on test data set: 0.79876


## Response == "hospitalized_or_IMV_or_death"

In [0]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

###############################################################################
## Current Y as response
## Notice: need manual check
## Possible options: hospitalized, invasive_mechanical_vent, death, results
################################################################################

select_col = 'hospitalized_or_IMV_or_death'
Y = train_df_Y_new[select_col]
# Y = Y.map(dict(yes=1, no=0))

class_names = np.unique(Y)
print("unique labels from y: ", class_names)

## Train test split use r_seed assigned in CMD 1
X = train_df_X
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=r_seed, stratify = Y)
columns = X_train.columns

## Normalize after the train/test split to avoid any potential data leakage
## using the min and max from the trainset to do minmax on test set
age_min, age_max = X_train["age"].min(), X_train["age"].max()
X_train["age"] = (X_train["age"] - age_min) / (age_max - age_min)
X_test["age"] = (X_test["age"] - age_min) / (age_max - age_min)

BMI_min, BMI_max = X_train["BMI"].min(), X_train["BMI"].max()
X_train["BMI"] = (X_train["BMI"] - BMI_min) / (BMI_max - BMI_min)
X_test["BMI"] = (X_test["BMI"] - BMI_min) / (BMI_max - BMI_min)

## Random oversampling using random seed
## Define oversampling strategy: default, 'auto': equivalent to 'not majority'
oversample = RandomOverSampler(random_state=r_seed)
os_data_X, os_data_y = oversample.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X, columns=columns)
os_data_y= pd.DataFrame(data=os_data_y, columns=['hospitalized_or_IMV_or_death'])


# define undersample strategy: “majority” will undersample the majority class determined by the class with the largest number of examples.
# undersample = RandomUnderSampler(sampling_strategy='majority', random_state=r_seed)
# us_data_X, us_data_y = undersample.fit_resample(X_train, y_train)
# us_data_X = pd.DataFrame(data=us_data_X, columns=columns)
# us_data_y= pd.DataFrame(data=us_data_y, columns=['hospitalized_or_IMV_or_death'])


# we can Check the numbers of our data
# print("length of oversampled data is ",len(os_data_X))
# print("Number of no hos in oversampled data",len(os_data_y[os_data_y['hospitalized_after_positive']==0]))
# print("Number of hos",len(os_data_y[os_data_y['hospitalized_after_positive']==1]))
# print("Proportion of no hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==0])/len(os_data_X))
# print("Proportion of hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==1])/len(os_data_X))

unique labels from y:  [0 1]


In [0]:
hos_IMV_death_result_df, os_LR_statmodel = multivariate_LR_model_result(os_data_X, os_data_y, X_train, y_train, opt_method)
display(hos_IMV_death_result_df)

Maximum Likelihood optimization failed to converge. Check mle_retvals


                                Logit Regression Results                                
Dep. Variable:     hospitalized_or_IMV_or_death   No. Observations:               190022
Model:                                    Logit   Df Residuals:                   189965
Method:                                     MLE   Df Model:                           56
Date:                          Wed, 03 May 2023   Pseudo R-squ.:                 0.09579
Time:                                  00:02:59   Log-Likelihood:            -1.1910e+05
converged:                                False   LL-Null:                   -1.3171e+05
Covariance Type:                      nonrobust   LLR p-value:                     0.000
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
age                              0.8758      0.018     49.459      0.000       0.841       0.9

Maximum Likelihood optimization failed to converge. Check mle_retvals

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Feature,Feature Importance,Odds ratio,lower,upper,Pvalue,Pvalue_FDR_corrected
age,0.9105073731615594,0.8758,0.841,0.911,0.0,0.0
BMI,2.3046726612445085,-0.4633,-3.231,2.305,0.564,1.0
sex:male,0.3432319713169585,-0.362,-0.381,-0.343,0.0,0.0
hypertension,0.07376782545738739,-0.1002,-0.127,-0.074,0.0,0.0
diabetes (type 1+2),0.34699802537678665,0.3164,0.286,0.347,0.0,0.0
atrial fibrillation,0.6442815026313677,0.6049,0.565,0.644,0.0,0.0
coronary artery disease,0.19650674821418193,0.1571,0.118,0.197,0.0,0.0
heart failure,0.7478368676351222,0.7074,0.667,0.748,0.0,0.0
chronic kidney disease,0.6720165588437568,0.6358,0.6,0.672,0.0,0.0
COPD,0.6996733985919064,0.6576,0.615,0.7,0.0,0.0


In [0]:
from sklearn.metrics import roc_auc_score
LR_model_binaryclass_auroc = binary_class_roc_auc_score(y_test, os_LR_statmodel.predict(X_test))

print("Auroc on test data set: %0.5f" % (LR_model_binaryclass_auroc))

Auroc on test data set: 0.70918


## Response == "IMV_or_death"

In [0]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

###############################################################################
## Current Y as response
## Notice: need manual check
## Possible options: hospitalized, invasive_mechanical_vent, death, results
################################################################################

select_col = 'IMV_or_death'
Y = train_df_Y_new[select_col]
# Y = Y.map(dict(yes=1, no=0))

class_names = np.unique(Y)
print("unique labels from y: ", class_names)

## Train test split use r_seed assigned in CMD 1
X = train_df_X
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=r_seed, stratify = Y)
columns = X_train.columns

## Normalize after the train/test split to avoid any potential data leakage
## using the min and max from the trainset to do minmax on test set
age_min, age_max = X_train["age"].min(), X_train["age"].max()
X_train["age"] = (X_train["age"] - age_min) / (age_max - age_min)
X_test["age"] = (X_test["age"] - age_min) / (age_max - age_min)

BMI_min, BMI_max = X_train["BMI"].min(), X_train["BMI"].max()
X_train["BMI"] = (X_train["BMI"] - BMI_min) / (BMI_max - BMI_min)
X_test["BMI"] = (X_test["BMI"] - BMI_min) / (BMI_max - BMI_min)

## Random oversampling using random seed
## Define oversampling strategy: default, 'auto': equivalent to 'not majority'
oversample = RandomOverSampler(random_state=r_seed)
os_data_X, os_data_y = oversample.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X, columns=columns)
os_data_y= pd.DataFrame(data=os_data_y, columns=['IMV_or_death'])


# define undersample strategy: “majority” will undersample the majority class determined by the class with the largest number of examples.
# undersample = RandomUnderSampler(sampling_strategy='majority', random_state=r_seed)
# us_data_X, us_data_y = undersample.fit_resample(X_train, y_train)
# us_data_X = pd.DataFrame(data=us_data_X, columns=columns)
# us_data_y= pd.DataFrame(data=us_data_y, columns=['IMV_or_death'])


# we can Check the numbers of our data
# print("length of oversampled data is ",len(os_data_X))
# print("Number of no hos in oversampled data",len(os_data_y[os_data_y['hospitalized_after_positive']==0]))
# print("Number of hos",len(os_data_y[os_data_y['hospitalized_after_positive']==1]))
# print("Proportion of no hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==0])/len(os_data_X))
# print("Proportion of hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==1])/len(os_data_X))

unique labels from y:  [0 1]


In [0]:
IMV_death_result_df, os_LR_statmodel = multivariate_LR_model_result(os_data_X, os_data_y, X_train, y_train, opt_method)
display(IMV_death_result_df)

Maximum Likelihood optimization failed to converge. Check mle_retvals


                           Logit Regression Results                           
Dep. Variable:           IMV_or_death   No. Observations:               212974
Model:                          Logit   Df Residuals:                   212917
Method:                           MLE   Df Model:                           56
Date:                Wed, 03 May 2023   Pseudo R-squ.:                  0.1633
Time:                        00:03:13   Log-Likelihood:            -1.2352e+05
converged:                      False   LL-Null:                   -1.4762e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
age                              0.7508      0.016     45.959      0.000       0.719       0.783
BMI                             -0.5792      1.309     -0.443      0.658     

Maximum Likelihood optimization failed to converge. Check mle_retvals

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Feature,Feature Importance,Odds ratio,lower,upper,Pvalue,Pvalue_FDR_corrected
age,0.7827855099512444,0.7508,0.719,0.783,0.0,0.0
BMI,1.9861136224089069,-0.5792,-3.144,1.986,0.417,1.0
sex:male,0.23809592688683057,-0.2565,-0.275,-0.238,0.0,0.0
hypertension,0.14758406960181567,-0.1728,-0.198,-0.148,0.0,0.0
diabetes (type 1+2),0.2093873217088023,0.1801,0.151,0.209,0.006,0.0879512510742147
atrial fibrillation,0.8069547917738142,0.7726,0.738,0.807,0.0,0.0
coronary artery disease,0.36309202749621705,0.3275,0.292,0.363,0.0,0.0
heart failure,0.9492697305717004,0.9142,0.879,0.949,0.0,0.0
chronic kidney disease,0.7337148000109475,0.7014,0.669,0.734,0.0,0.0
COPD,0.7250482265047398,0.6872,0.649,0.725,0.0,0.0


In [0]:
from sklearn.metrics import roc_auc_score
LR_model_binaryclass_auroc = binary_class_roc_auc_score(y_test, os_LR_statmodel.predict(X_test))

print("Auroc on test data set: %0.5f" % (LR_model_binaryclass_auroc))

Auroc on test data set: 0.76504
